In [1]:
import os
import matplotlib.pyplot as plt
%matplotlib qt
os.getcwd()
os.chdir('/home/wouter/pptis_analysis/repptis1/')

QSocketNotifier: Can only be used with threads started with QThread


In [2]:
from tistools import *
from pprint import pprint
import logging 
logging.basicConfig(level=logging.INFO)
inputfile = "./retis.rst"
indir = "./"
folders = glob.glob("./0[0-9][0-9]")
folders = sorted(folders)
interfaces, zero_left, timestep = read_inputfile(inputfile)
LMR_interfaces, LMR_strings = get_LMR_interfaces(interfaces, zero_left)
pathensembles = []
for i,fol in enumerate(folders):
    print(fol)
    pe = read_pathensemble(fol+"/pathensemble.txt")
    pe.set_name(fol[len(indir):])
    pe.set_interfaces([LMR_interfaces[i], LMR_strings[i]])
    if i==0:
        pe.set_zero_minus_one = True
        pe.set_in_zero_minus_one = True
    if i==1:
        pe.set_in_zero_plus = True 
    w, _ = get_weights(pe.flags, ACCFLAGS, REJFLAGS, verbose = False)
    pe.set_weights(w)
    print("pathensemble info: ")
    pprint(vars(pe))
    pathensembles.append(pe)

./000
pathensemble info: 
{'cyclenumbers': array([     0,      1,      2, ...,  99998,  99999, 100000]),
 'flags': array(['ACC', 'ACC', 'ACC', ..., 'FTL', 'BTL', 'FTL'], dtype='<U3'),
 'generation': array(['ld', 'sh', 'sh', ..., 'sh', 'sh', 'sh'], dtype='<U2'),
 'has_zero_minus_one': False,
 'in_zero_minus': False,
 'in_zero_plus': False,
 'interfaces': [[0.1, 0.15000000000000002, 0.2],
                ['l_[-1]', '( l_[-1] + l_[0] ) / 2', 'l_[0]']],
 'lambmaxs': array([0.18995829, 0.20115691, 0.20129873, ..., 0.16037035, 0.18323515,
       0.17180941]),
 'lambmins': array([0.08995829, 0.17135828, 0.0973619 , ..., 0.09888888, 0.12286248,
       0.09614375]),
 'lengths': array([  4,  69, 104, ...,  81,  62,  67]),
 'lmrs': array(['LM*', 'R*R', 'LMR', ..., 'LM*', '*M*', 'LM*'], dtype='<U3'),
 'name': '000',
 'ncycle': 100001,
 'newpathnumbers': array([    0,     1,     2, ..., 44278, 44278, 44278]),
 'pathnumbers': array([    1,     2,     3, ..., 51461, 51461, 51461]),
 'set_in_zero_minu

In [3]:
# set matplotlib parameters 
import matplotlib as mpl
mpl.rcParams['font.size'] = 12
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['axes.labelsize'] = 12
mpl.rcParams['axes.titlesize'] = 12
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['figure.titlesize'] = 12

In [4]:
# Analyze the pptis simulation. Block averages, running averages, etc.
# Analysis output is saved to the data dictionary, which is pickled at the end.
data = {}
for i, pe in enumerate(pathensembles):
    print("doing pathensemble {}".format(i))
    if i == 0:
        continue  # TODO: make [0-] analysis. 
    masks = get_lmr_masks(pe)
    loadmask = get_generation_mask(pe, "load")
    #hardloadmask = get_hard_load_mask(loadmask)
    accmask = get_flag_mask(pe, "ACC")
    pathtypes = ("LML", "LMR", "RML", "RMR")
    pathtype_cycles = {}
    for ptype in pathtypes:
        pathtype_cycles[ptype] = unwrap_by_weight(
                (pe.lmrs == ptype).astype(int), pe.weights)
            #select_with_masks(np.all([~loadmask, masks[ptype]], 
            #                         axis=0).astype(int), [~hardloadmask])
    data[i] = {}
    data[i]["running"] = {}
    data[i]["running"]["plocal"] = {}
    # you'll still have to hardload select pe.weigths... TODO
    for (ptype, p_loc) in zip(pathtypes, 
                              running_avg_local_probs(pathtype_cycles, 
                                                      pe.weights, tr = True)):
        data[i]["running"]["plocal"][ptype] = p_loc
    plocfull = get_local_probs(pe, tr=True)
    data[i]["full"] = {}
    for ptype in pathtypes:
        data[i]["full"][ptype] = plocfull[ptype]
    
    # block analysis, not yet loadpath aware TODO
    u = 1*(pe.lmrs == "LML").astype(int) \
        + 2*(pe.lmrs == "LMR").astype(int) \
        - 1*(pe.lmrs == "RMR").astype(int) \
        - 2*(pe.lmrs == "RML").astype(int)
    u = unwrap_by_weight(u, pe.weights)
    opt_block_size, opt_tau_int, opt_num_ind, opt_mean, opt_se_mean, \
        block_sizes, inefficiencies, se_means = full_block_analysis(u)
    #opt_block_size = min([int(10*opt_block_size), len(pe.lmrs)//2])
    #opt_block_size = len(pe.lmrs)//10
    fig, ax = plt.subplots()
    ax.plot(block_sizes, se_means, lw=0.5, c="k")
    ax.set_xlabel('Block size')
    ax.set_ylabel('Standard error')
    #ax.semilogx()
    ax.axvline(opt_block_size, color='r', linestyle='--', lw = 0.5)
    ax.axhline(opt_se_mean, color='r', linestyle='-', lw = 0.5)
    # put the optimal block size and standard error in the title 
    title =r"$N_{{blocks}}^{{opt}}$ = {:.0f}, $\sigma_{{SE}}$ = {:.6f}".format(
        opt_block_size, opt_se_mean)
    ax.set_title(title)
    fig.savefig(f"pathensemble_{i}_block.pdf")
    plt.close(fig)
    #print("optimal block size: {}".format(opt_block_size))
    opt_block_size = 10000
    data[i]["blocksize"] = opt_block_size
    opt_block_size = int(opt_block_size)
    # determine the number of blocks that can be formed
    num_blocks = np.floor(len(pe.lmrs) / opt_block_size).astype(int)
    print("N_blocks = {}, N_data = {}, block_size = {}".format(
        num_blocks, len(pe.lmrs), opt_block_size)
    )
    # trim x to have a multiple of block_size
    cyc_block = np.arange(len(pe.lmrs))[:num_blocks * opt_block_size]
    # reshape x to a matrix of blocks
    cyc_block = cyc_block.reshape(num_blocks, opt_block_size)
    data[i]["blocks"] = {"LML": [], "LMR": [], "RML": [], "RMR": []}
    data[i]["numblocks"] = num_blocks
    # First we unify the pathensemble, such taht we don't get the following 
    # problem: if we have a block of [0, 0, 0, newLMR, ...], then the first
    # three zeros may correspond to an earlier LMR, and these are not accounted
    # for in the current block...
    upe = pe.unify_pe()
    for cycnums in cyc_block:
        #print("doing blocklen {}".format(len(cycnums)))
        pe_block = upe.sample_pe(cycnums)
        ploc = get_local_probs(pe_block, w = pe_block.weights, tr = True)
        for ptype in pathtypes:
            data[i]["blocks"][ptype].append(ploc[ptype])
    # calculate the mean and standard error of the mean for each block
    data[i]["meanploc"] = {}
    data[i]["semploc"] = {}
    for ptype in pathtypes:
        data[i]["meanploc"][ptype] = np.nanmean(data[i]["blocks"][ptype], axis=0)
        data[i]["semploc"][ptype] = np.nanstd(data[i]["blocks"][ptype], axis=0) \
            / np.sqrt(num_blocks - 1)
        
    # Cross distances distribution
    makefigs = True 
    if makefigs: 
        L, M, R, lmlpercs, lmllambs, rmrpercs, rmrlambs = cross_dist_distr(upe)
        fig,ax =plt.subplots()
        ax.plot(lmllambs, lmlpercs, lw=1, c="g")
        ax.plot(rmrlambs, rmrpercs, lw=1, c="r")
        for lamb in (L,M,R):
            ax.axvline(lamb, color='k', linestyle='--', lw = 0.5)
        ax.set_xlabel('Cross distance')
        ax.set_ylabel('Frequency')
        ax.set_title("Ensemble {}. L = {}, M = {}, R = {}".format(
            pe.name, L, M, R))
        ax.set_ylim(0)
        fig.savefig(f"pathensemble_{i}_crossdist.pdf")
        plt.close(fig)

        # Pathlength distribution
        data[i]["pathlengths"] = pathlength_distr(upe)
        for ptype in pathtypes:
            fig, ax = plt.subplots()
            ax.plot(data[i]["pathlengths"][ptype]["bin_centers"], 
                data[i]["pathlengths"][ptype]["hist"])
            ax.set_xlabel('Pathlength')
            ax.set_ylabel('Frequency')
            ax.set_title(f"{np.sum(data[i]['pathlengths'][ptype]['hist'])} " + \
                         f"{ptype} paths. ")
            ax.legend([f"mean = {data[i]['pathlengths'][ptype]['mean']:.2f}, " + \
                          f"std = {data[i]['pathlengths'][ptype]['std']:.2f}"])
            fig.savefig(f"pathensemble_{i}_pathlength_{ptype}.pdf")
            plt.close(fig)


doing pathensemble 0
doing pathensemble 1
Weights of the different paths:
wRMR = 0
wRML = 8268
wLMR = 8311
wLML = 83421
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 0.09038768734223453
pLML = 0.9096123126577654


/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/tistools/repptis_analysis.py:553: RuntimeWarning: invalid value encountered in divide
  p_NP = cumsums['LMR']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/tistools/repptis_analysis.py:554: RuntimeWarning: invalid value encountered in divide
  p_NN = cumsums['LML']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/tistools/repptis_analysis.py:555: RuntimeWarning: invalid value encountered in divide
  p_PP = cumsums['RMR']/(cumsums['RML']+cumsums['RMR'])
/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/tistools/repptis_analysis.py:556: RuntimeWarning: invalid value encountered in divide
  p_PN = cumsums['RML']/(cumsums['RML']+cumsums['RMR'])


N_blocks = 10, N_data = 100001, block_size = 10000
Are all weights 1?  True
Are all paths accepted?  True
Weights of the different paths:
wRMR = 0
wRML = 1000
wLMR = 934
wLML = 8065
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 0.10706377325066431
pLML = 0.8929362267493357
Weights of the different paths:
wRMR = 0
wRML = 843
wLMR = 766
wLML = 8391
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 0.08748844543526725
pLML = 0.9125115545647328
Weights of the different paths:
wRMR = 0
wRML = 784
wLMR = 852
wLML = 8364
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 0.08908734480505337
pLML = 0.9109126551949467
Weights of the different paths:
wRMR = 0
wRML = 846
wLMR = 800
wLML = 8354
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 0.08968072354800044
pLML = 0.9103192764519996
Weights of the different paths:
wRMR = 0
wRML = 791
wLMR = 900
wLML = 8309
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 0.09235894915069091
pLML = 0.9076410508

/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


doing pathensemble 2
Weights of the different paths:
wRMR = 167
wRML = 4373
wLMR = 4262
wLML = 91198
Local crossing probabilities:
pRMR = 0.037239380086966214
pRML = 0.9627606199130337
pLMR = 0.04520208761928692
pLML = 0.9547979123807131


...

N_blocks = 10, N_data = 100001, block_size = 10000
Are all weights 1?  True
Are all paths accepted?  True
Weights of the different paths:
wRMR = 1527
wRML = 1455
wLMR = 1430
wLML = 5585
Local crossing probabilities:
pRMR = 0.5142279845091766
pRML = 0.4857720154908234
pLMR = 0.20526503023834936
pLML = 0.7947349697616507
Weights of the different paths:
wRMR = 1531
wRML = 1599
wLMR = 1550
wLML = 5320
Local crossing probabilities:
pRMR = 0.4929962968926099
pRML = 0.5070037031073901
pLMR = 0.22837044020596128
pLML = 0.7716295597940387
Weights of the different paths:
wRMR = 1283
wRML = 1476
wLMR = 1328
wLML = 5913
Local crossing probabilities:
pRMR = 0.47783985102420856
pRML = 0.5221601489757914
pLMR = 0.19166097060833903
pLML = 0.808339029391661
Weights of the different paths:
wRMR = 1657
wRML = 1603
wLMR = 1619
wLML = 5121
Local crossing probabilities:
pRMR = 0.5070379436964504
pRML = 0.49296205630354956
pLMR = 0.2393048128342246
pLML = 0.7606951871657754
Weights of the different paths:
wR

/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/tistools/repptis_analysis.py:553: RuntimeWarning: invalid value encountered in divide
  p_NP = cumsums['LMR']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/tistools/repptis_analysis.py:554: RuntimeWarning: invalid value encountered in divide
  p_NN = cumsums['LML']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/tistools/repptis_analysis.py:555: RuntimeWarning: invalid value encountered in divide
  p_PP = cumsums['RMR']/(cumsums['RML']+cumsums['RMR'])
/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/tistools/repptis_analysis.py:556: RuntimeWarning: invalid value encountered in divide
  p_PN = cumsums['RML']/(cumsums['RML']+cumsums['RMR'])


N_blocks = 10, N_data = 100001, block_size = 10000
Are all weights 1?  True
Are all paths accepted?  True
Weights of the different paths:
wRMR = 1279
wRML = 2664
wLMR = 2737
wLML = 3318
Local crossing probabilities:
pRMR = 0.3213971604472924
pRML = 0.6786028395527076
pLMR = 0.44869984215336045
pLML = 0.5513001578466395
Weights of the different paths:
wRMR = 1556
wRML = 2696
wLMR = 2837
wLML = 2911
Local crossing probabilities:
pRMR = 0.35997686524002315
pRML = 0.6400231347599769
pLMR = 0.48727432848965213
pLML = 0.5127256715103479
Weights of the different paths:
wRMR = 1364
wRML = 2601
wLMR = 2737
wLML = 3298
Local crossing probabilities:
pRMR = 0.33820976940242997
pRML = 0.66179023059757
pLMR = 0.4472934472934473
pLML = 0.5527065527065527
Weights of the different paths:
wRMR = 1417
wRML = 2751
wLMR = 2848
wLML = 2984
Local crossing probabilities:
pRMR = 0.336060713862208
pRML = 0.663939286137792
pLMR = 0.48404945102446617
pLML = 0.5159505489755338
Weights of the different paths:
wRMR 

/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/tistools/repptis_analysis.py:553: RuntimeWarning: invalid value encountered in divide
  p_NP = cumsums['LMR']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/tistools/repptis_analysis.py:554: RuntimeWarning: invalid value encountered in divide
  p_NN = cumsums['LML']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/tistools/repptis_analysis.py:555: RuntimeWarning: invalid value encountered in divide
  p_PP = cumsums['RMR']/(cumsums['RML']+cumsums['RMR'])
/home/wouter/anaconda/envs/tistool/lib/python3.11/site-packages/tistools/repptis_analysis.py:556: RuntimeWarning: invalid value encountered in divide
  p_PN = cumsums['RML']/(cumsums['RML']+cumsums['RMR'])


N_blocks = 10, N_data = 100001, block_size = 10000
Are all weights 1?  True
Are all paths accepted?  True
Weights of the different paths:
wRMR = 905
wRML = 1835
wLMR = 1748
wLML = 5511
Local crossing probabilities:
pRMR = 0.3356202484702392
pRML = 0.6643797515297608
pLMR = 0.2453269428277987
pLML = 0.7546730571722013
Weights of the different paths:
wRMR = 862
wRML = 1771
wLMR = 1843
wLML = 5524
Local crossing probabilities:
pRMR = 0.32296740352191833
pRML = 0.6770325964780817
pLMR = 0.2464875187559678
pLML = 0.7535124812440321
Weights of the different paths:
wRMR = 832
wRML = 1808
wLMR = 1776
wLML = 5584
Local crossing probabilities:
pRMR = 0.3170731707317073
pRML = 0.6829268292682927
pLMR = 0.24295010845986983
pLML = 0.7570498915401301
Weights of the different paths:
wRMR = 844
wRML = 1747
wLMR = 1976
wLML = 5433
Local crossing probabilities:
pRMR = 0.31195712437627054
pRML = 0.6880428756237295
pLMR = 0.2551922681472342
pLML = 0.7448077318527658
Weights of the different paths:
wRMR = 

In [5]:
for i, _ in enumerate(pathensembles):
    if i == 0:
        continue # TODO: add 0- ensemble analysis

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
    ax1.plot(data[i]["running"]["plocal"]["LMR"], label="LMR", lw=1,
            c="green", ls="-")
    # plot mean value axhline, and 2 standard error lines
    ax1.axhline(data[i]["meanploc"]["LMR"], color="green", linestyle="--",
                label=r"$\langle p^{{\pm}} \rangle$")
    ax1.axhline(data[i]["meanploc"]["LMR"] + 2*data[i]["semploc"]["LMR"],
                color="green", linestyle=":",
                label=r"$\langle p^{{\pm}} \rangle \pm 2\sigma$")
    ax1.axhline(data[i]["meanploc"]["LMR"] - 2*data[i]["semploc"]["LMR"],
                color="green", linestyle=":",)
    ax1.set_ylabel(r"$p^{{\pm}}$")
    # also plot the full data for LMR, axhline, and plot the value next to it
    ax1.axhline(data[i]["full"]["LMR"], color="k", linestyle="-",
                lw=1, label = data[i]["full"]["LMR"])
    # nice x axis labels, scientific notation
    ax1.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    ax1.legend()
    ax2.plot(data[i]["running"]["plocal"]["RML"], label="RML", lw=1,
            c="red", ls="-")
    ax2.axhline(data[i]["meanploc"]["RML"], color="red", linestyle="--",
                    label=r"$\langle p^{{\mp}} \rangle$")
    ax2.axhline(data[i]["meanploc"]["RML"] + 2*data[i]["semploc"]["RML"],
                    color="red", linestyle=":", 
                    label=r" $\langle p^{{\mp}} \rangle \pm 2\sigma$")
    ax2.axhline(data[i]["meanploc"]["RML"] - 2*data[i]["semploc"]["RML"],
                    color="red", linestyle=":",)
    ax2.set_xlabel("Cycle")
    ax2.set_ylabel(r"$p^{{\mp}}$")
    ax2.axhline(data[i]["full"]["RML"], color="k", linestyle="-",
                lw=1, label = data[i]["full"]["RML"])
    ax2.legend()
    fig.suptitle("Pathensemble {}: Local cross probs".format(i))
    fig.tight_layout()
    #fig.show()
    fig.savefig("pathensemble_{}_running.pdf".format(i))
    plt.close(fig)

    # make histogram of LMR and RML values
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=False)
    ax1.hist(data[i]["blocks"]["LMR"], bins=50, density=True,
            label="LMR", color="green")
    ax1.axvline(data[i]["meanploc"]["LMR"], color="k", linestyle="-", lw = 1,
                label=r"$\langle p^{{\pm}} \rangle$")
    ax1.axvline(data[i]["meanploc"]["LMR"] + 2*data[i]["semploc"]["LMR"],
                    color="k", linestyle=":", 
                    label=r"$\langle p^{{\pm}}\rangle\pm2\sigma$")
    ax1.axvline(data[i]["meanploc"]["LMR"] - 2*data[i]["semploc"]["LMR"],
                    color="k", linestyle = ":")  
    ax1.set_ylabel("Frequency")
    ax1.legend()
    ax2.hist(data[i]["blocks"]["RML"], bins=50, density=True,
            label="RML", color="red")
    ax2.axvline(data[i]["meanploc"]["RML"], color="k", linestyle="-", lw =1,
                label=r"$\langle p^{{\mp}} \rangle$")
    ax2.axvline(data[i]["meanploc"]["RML"] + 2*data[i]["semploc"]["RML"],
                    color="k", linestyle=":", 
                    label=r"$\langle p^{{\mp}} \rangle  \pm2\sigma$")
    ax2.axvline(data[i]["meanploc"]["RML"] - 2*data[i]["semploc"]["RML"],
                    color="k", linestyle = ":")
    ax2.set_xlabel("p")
    ax2.set_ylabel("Frequency")
    ax2.legend()
    fig.suptitle("Pathensemble {}: block hist. ".format(i)+\
                 r"$N_b = {}$".format(data[i]["numblocks"]))
    fig.tight_layout()
    #fig.show()
    fig.savefig("pathensemble_{}_hist.pdf".format(i))
    plt.close(fig)

In [14]:
# Global crossing probabilities

# WITHOUT ERRORS #  
# Full data 
psfull = []
for i in range(1, len(pathensembles)):
    psfull.append({"LMR": data[i]["full"]["LMR"], 
               "RML": data[i]["full"]["RML"], 
               "RMR": data[i]["full"]["RMR"],
               "LML": data[i]["full"]["LML"]})
    
Pminfull, Pplusfull, Pcrossfull = get_globall_probs(psfull)


# WITH ERRORS #

probs = []
for i in range(1, len(pathensembles)):
    probs.append({"LMR": {"mean": data[i]["meanploc"]["LMR"], 
                           "std": data[i]["semploc"]["LMR"]}, 
                   "RML": {"mean": data[i]["meanploc"]["RML"], 
                           "std": data[i]["semploc"]["RML"]}, 
                   "RMR": {"mean": data[i]["meanploc"]["RMR"], 
                           "std": data[i]["semploc"]["RMR"]},
                   "LML": {"mean": data[i]["meanploc"]["LML"], 
                           "std": data[i]["semploc"]["LML"]}})

Pm, Pp, Pc, Pme, Ppe, Pce = get_global_probs(probs)


# Finally, we also calculate global probs per block. (splitting entire 
# simulation in 10 blocks, and then ...)
list_of_probs = []
for i in range(10):
    problist = []
    for j in range(1,len(pathensembles)):
        dic = {"LMR": data[j]["blocks"]["LMR"][i],
               "RML": data[j]["blocks"]["RML"][i],
               "RMR": data[j]["blocks"]["RMR"][i],
               "LML": data[j]["blocks"]["LML"][i]}
        problist.append(dic)
    list_of_probs.append(problist)
Pmins, Ppluss, Pcrosss = [], [], []
for el in list_of_probs:
    Pmin, Pplus, Pcross = get_globall_probs(el)
    Pmins.append(Pmin)
    Ppluss.append(Pplus)
    Pcrosss.append(Pcross)
Pcross_avg = np.mean(Pcrosss, axis=0)
Pcross_std = np.std(Pcrosss, axis=0)
Pcross_err = Pcross_std / np.sqrt(len(Pcrosss))

# Make a figure of the global crossing probabilities
fig, ax = plt.subplots()
ax.set_yscale("log")
ax.plot(Pcrossfull, "o", c = "r", label = "full")
ax.errorbar(np.arange(len(Pc)), Pc, yerr=Pce, fmt="x", c="b", label = "block propagated")
ax.errorbar(np.arange(len(Pc)), Pcross_avg, yerr=Pcross_err, fmt="v", label = "split block avg")
ax.set_xlabel("intf")
ax.legend()
ax.set_ylabel(r"$P_A(\lambda_i|\lambda_A)$")
fig.tight_layout()
fig.show()
fig.savefig("Global_probs.pdf")

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [15]:
# Compile into a latex document
with open("pathensemble.tex", "w") as f:
    f.write(r"\documentclass[12pt]{article}" + " \n")
    f.write(r"\usepackage[margin=0.1in]{geometry}" + " \n")
    f.write(r"\usepackage{amsmath}" + " \n")
    f.write(r"\usepackage{amssymb}" + " \n")
    f.write(r"\usepackage{amsfonts}" + " \n")
    f.write(r"\usepackage{amsthm}" + " \n")
    f.write(r"\usepackage{bm}" + " \n")
    f.write(r"\usepackage{color}" + " \n")
    f.write(r"\usepackage{hyperref}" + " \n")
    f.write(r"\usepackage{float}" + " \n")
    f.write(r"\usepackage{tikz}" + " \n")
    f.write(r"\usepackage{pgfplots}" + " \n")
    f.write(r"\usepackage{pgfplotstable}" + " \n")
    f.write(r"\usepackage{pgf}" + " \n")
    f.write(r"\usepackage{caption}" + " \n")
    f.write(r"\usepackage{setspace}" + " \n")
    f.write(r"\usepackage{float}" + " \n")
    f.write(r"\usepackage{graphicx}" + " \n")

    u = 1*(pe.lmrs == "LML").astype(int) \
        + 2*(pe.lmrs == "LMR").astype(int) \
        - 1*(pe.lmrs == "RMR").astype(int) \
        - 2*(pe.lmrs == "RML").astype(int)

    f.write(r"\begin{document}" + " \n")
    f.write(r"\section{Information}" + " \n")
    f.write(r"Figures dealing with local crossing probabilities take into ")
    f.write(r"account the `infinite' time-reversal moves. ")
    f.write(r"Figures dealing with path lengths do not include this, so the ")
    f.write(r"amount of LMR and RML paths are different. ")
    f.write(r"For each path ensemble, the following is performed/shown:" +"\n\n")
    f.write(r"Block analysis is performed, where the optimal block size is ")
    f.write(r"determined to minimize the statistical inefficiency on the signal ")
    f.write(r"$S(n) = 1I_{LML}(n) + 2I_{LMR}(n) - 1I_{RMR}(n) - 2I_{RML}(n)$, where ")
    f.write(r"$I_{X}(n)$ is the indicator function for the X state. For example, ")
    f.write(r"if cycle $n$ holds an LML path, then $I_{LML}(n)=1$, and ")
    f.write(r"$I_{LMR}(n)=I_{RMR}(n)=I_{RML}(n)=0$. ")
    f.write(r"I noticed that taking separate functions for each state, and then ")
    f.write(r"taking the most restraining optimal block size, gives roughly the same result. ")
    f.write(r"The mean value $\langle \bar{p}^{\pm} \rangle$ is the mean of ")
    f.write(r"the block averages, where $\bar{p}$ denotes a block mean, and ")
    f.write(r"$\langle \rangle$ denotes the average over all blocks. ")
    f.write(r"The standard error of the mean $\sigma$ is the standard ")
    f.write(r"deviation of the block averages, divided by the square ")
    f.write(r"root of the number of blocks"+ " \n\n")
    f.write(r"One would assume that the mean of block averages would be the ")
    f.write(r"same as the mean of all the data combined, but this is not the ")
    f.write(r"case. This is because $\bar{p}^\pm=N_{LMR}/(N_{LMR}+N_{LML})$ ")
    f.write(r"is not linear in $N_{LMR}$ and $N_{LML}$. Therefore, the average ")
    f.write(r"over the blocks is not the same as the limit of the running avg. ")
    f.write(r"This holds true for the other local crossing probabilities."+ " \n\n")
    f.write(r"On the local crossing probability plots, the running average is ")
    f.write(r"shown. The black horizontal line denotes the final value. The ")
    f.write(r"dashed lines denote the mean, and the mean $\pm$ 2 standard ")
    f.write(r"errors. These dashed lines are also given on the histogram plots ")
    f.write(r"on the right, using vertical lines.")
    f.write(r"\newpage" + " \n")

    # Tabulate the interfaces for each ensemble
    f.write(r"\section{Interfaces}" + " \n")
    f.write(r"\begin{table}[H]" + " \n")
    f.write(r"\centering" + " \n")
    f.write(r"\begin{tabular}{|c|c|c|c|}" + " \n")
    f.write(r"\hline" + " \n")
    f.write(r"Ensemble & $L$ & $M$ & $R$ \\" + " \n")
    f.write(r"\hline" + " \n")
    for i in range(1, len(pathensembles)):
            f.write(r"{} & ".format(i) + " \n")
            f.write(r"{} & ".format(pathensembles[i].interfaces[0][0]) + " \n")
            f.write(r"{} & ".format(pathensembles[i].interfaces[0][1]) + " \n")
            f.write(r"{} \\".format(pathensembles[i].interfaces[0][2]) + " \n")
            f.write(r"\hline" + " \n")
    f.write(r"\end{tabular}" + " \n")
    f.write(r"\caption{Interfaces for each ensemble.}" + " \n")
    f.write(r"\end{table}" + " \n")
    

    # Tabulate the local crossing probabilities for each ensemble
    f.write(r"\section{Local crossing probabilities}" + " \n")
    f.write(r"\begin{table}[H]" + " \n")
    f.write(r"\centering" + " \n")
    f.write(r"\begin{tabular}{|c|c|c|c|c|c|c|c|c|}" + " \n")
    f.write(r"\hline" + " \n")
    f.write(r"Ensemble & $\langle p^{\pm} \rangle_{\text{full}}$ & ")
    f.write(r"$\langle p^{\mp} \rangle_{\text{full}}$ & ")
    f.write(r"$\langle p^{\pm} \rangle_{\text{block}}$ & ")
    f.write(r"$\langle p^{\mp} \rangle_{\text{block}}$ & ")
    f.write(r"$\sigma_{\text{block}}^{\pm}$ & ")
    f.write(r"$\sigma_{\text{block}}^{\mp}$ & ")
    f.write(r"$N_{\text{blocks}}$ & ")
    f.write(r"$N_{\text{cycles}}$ \\" + " \n")
    f.write(r"\hline" + " \n")
    for i in range(1, len(pathensembles)):
            f.write(r"{} & ".format(i) + " \n")
            f.write(r"{:.6f} & ".format(data[i]["full"]["LMR"]) + " \n")
            f.write(r"{:.6f} & ".format(data[i]["full"]["RML"]) + " \n")
            f.write(r"{:.6f} & ".format(data[i]["meanploc"]["LMR"]) + " \n")
            f.write(r"{:.6f} & ".format(data[i]["meanploc"]["RML"]) + " \n")
            f.write(r"{:.6f} & ".format(data[i]["semploc"]["LMR"]) + " \n")
            f.write(r"{:.6f} & ".format(data[i]["semploc"]["RML"]) + " \n")
            f.write(r"{} & ".format(data[i]["numblocks"]) + " \n")
            f.write(r"{} \\".format(len(pathensembles[i].lmrs)) + " \n")
            f.write(r"\hline" + " \n")
    f.write(r"\end{tabular}" + " \n")
    f.write(r"\caption{Local crossing probabilities for each ensemble. " +
            r"$\langle p^{\pm} \rangle$ and $\langle p^{\mp} \rangle$ are the " +
            r"block-avgeraged means, $\langle p^{\pm} \rangle_{\text{full}}$ and " +
            r"$\langle p^{\mp} \rangle_{\text{full}}$ are the full data " +
            r"averages. $\sigma_{\text{block}}^{+}$ and " +
            r"$\sigma_{\text{block}}^{-}$ are the standard errors of the " +
            r"block-averaged means. $N_{\text{blocks}}$ is the number of blocks, " +
            r"and $N_{\text{cycle}}$ is the number of cycles. The block-averaged " +
            r"mean may differ from the full-data mean, because of non-linearity.}" + " \n")
    f.write(r"\end{table}" + " \n")

    # Include the global cross probabilities plot
    f.write(r"\section{Global cross probabilities}" + " \n")
    f.write(r"\begin{figure}[H]" + " \n")
    f.write(r"\centering" + " \n")
    f.write(r"\includegraphics[width=0.49\textwidth]{" +
            "Global_probs.pdf" + "}" + " \n")
    f.write(r"\end{figure}" + " \n")
    f.write(r"\newpage" + " \n")

    # Don't use subfigures, just use figures of widht 0.49\textwidth
    for i in range(1, len(pathensembles)):
        f.write(r"\begin{figure}[H]" + " \n")
        f.write(r"\centering" + " \n")
        f.write(r"\includegraphics[width=0.49\textwidth]{" + 
                "pathensemble_{}_running.pdf".format(i) + "}" + " \n")
        f.write(r"\includegraphics[width=0.49\textwidth]{" + 
                "pathensemble_{}_hist.pdf".format(i) + "}" + " \n")
        f.write(r"\includegraphics[width=0.49\textwidth]{" +
                "pathensemble_{}_block.pdf".format(i) + "}" + " \n")
        f.write(r"\includegraphics[width=0.49\textwidth]{" +
                "pathensemble_{}_crossdist.pdf".format(i) + "}" + " \n")
        for ptype in ("LML", "LMR", "RML", "RMR"):
            f.write(r"\includegraphics[width=0.333\textwidth]{" +
                    "pathensemble_{}_pathlength_{}.pdf".format(i, ptype) + "}" +
                      " \n")
        f.write(r"\end{figure}" + " \n")
    f.write(r"\end{document}" + " \n")
    
